# Tutorial 4: Continuous Functions

*Scott Continuity and Computability*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-continuous-domains/blob/main/notebooks/04-continuous-functions.ipynb)

---

## A Note from the Archives

> *Year 865, Capital Archives*
>
> Arren Mott had established the structure of domains. Now he needed to characterize which functions between domains were "well-behaved."
>
> "Not every function respects the information ordering," Mott wrote. "Consider a function that returns 1 if its input terminates, and 0 otherwise. This function examines *infinite* behavior — it requires complete information about its input before producing any output."
>
> "The functions we care about," he continued, "are those that produce output based on *finite* input. If you give me more information, I give you more information. If you give me a limit, I give you the limit."
>
> This was EV-865-001: the discovery of Scott continuity.

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Define monotonicity for functions on domains
2. Explain Scott continuity and why it captures computability
3. Identify continuous and non-continuous functions
4. Verify continuity by checking limit preservation
5. Understand why parallel or is not continuous

## Setup

In [ ]:
import pandas as pd
import numpy as np

# Load datasets
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

functions_df = pd.read_csv(BASE_URL + "continuous_functions.csv")

print(f"Loaded {len(functions_df)} function examples")

## 1. Monotonicity

A function f is **monotone** if:

$$x \sqsubseteq y \implies f(x) \sqsubseteq f(y)$$

**More information in → more information out.**

This is the minimum requirement for a "reasonable" function on domains.

In [ ]:
# All functions in our dataset
print("Functions and their monotonicity:")
print()
print(functions_df[['function_name', 'notation', 'is_monotone']].to_string(index=False))

## 2. Scott Continuity

A function f is **Scott-continuous** if it preserves directed limits:

$$f(\bigsqcup D) = \bigsqcup \{f(d) \mid d \in D\}$$

**The function of the limit equals the limit of the functions.**

This means:
- f only needs finite information to produce output
- f doesn't "jump" at limits
- f is effectively computable

In [ ]:
# Continuous functions
continuous = functions_df[functions_df['is_continuous'] == True]

print("Continuous functions (preserve directed limits):")
print()
for _, row in continuous.head(10).iterrows():
    print(f"  {row['function_name']}: {row['notation']}")
    print(f"      {row['notes']}")
    print()

## 3. Example: Successor is Continuous

The successor function succ(n) = n + 1 is continuous.

**Why?** Consider a chain of approximations:
```
⊥ ⊑ 3 (a constant chain ending at 3)
```

Then:
```
succ(⊔{⊥, 3}) = succ(3) = 4
⊔{succ(⊥), succ(3)} = ⊔{⊥, 4} = 4
```

They're equal! The function preserves the limit.

In [ ]:
# Demonstrate successor continuity
def succ(n):
    """Successor function (strict in its argument)."""
    if n == "⊥":
        return "⊥"  # succ(⊥) = ⊥
    return n + 1

print("Successor function:")
print(f"  succ(⊥) = {succ('⊥')}")
print(f"  succ(0) = {succ(0)}")
print(f"  succ(5) = {succ(5)}")
print()
print("Continuity check:")
print("  Chain: ⊥ ⊑ 5")
print(f"  succ(⊔{{⊥, 5}}) = succ(5) = {succ(5)}")
print(f"  ⊔{{succ(⊥), succ(5)}} = ⊔{{⊥, 6}} = 6")
print("  Equal! ✓")

## 4. Non-Example: Parallel Or

The **parallel or** function is NOT continuous.

Parallel or returns true if either argument is true, without waiting for both to terminate.

```
por(true, ⊥) = true
por(⊥, true) = true
por(false, false) = false
```

This function "looks inside" non-termination — it needs to check both arguments simultaneously.

In [ ]:
# Parallel or is not continuous
non_continuous = functions_df[functions_df['is_continuous'] == False]

print("Non-continuous functions:")
print()
for _, row in non_continuous.iterrows():
    print(f"  {row['function_name']}: {row['notation']}")
    print(f"      {row['notes']}")
    print()

## 5. Why Continuity = Computability

Continuous functions are exactly the **effectively computable** functions.

**Intuition**: A computable function can only examine a finite amount of its input before producing output. It cannot:
- Wait for non-terminating computations to finish
- Check "does this program halt?"
- Examine infinite lists completely

This is why parallel or is not computable by a sequential machine.

In [ ]:
# Finite information principle
print("The Finite Information Principle:")
print()
print("  If f is continuous and f(x) = y ≠ ⊥, then:")
print("  There exists a FINITE approximation x' ⊑ x such that f(x') = y.")
print()
print("  This means f only needed to look at finite information!")
print()
print("  Example: head([1,2,3,...]) = 1")
print("  The function only needs to see [1:⊥], not the whole infinite list.")

## 6. Strictness

A function f is **strict** if f(⊥) = ⊥.

Most arithmetic functions are strict:
- succ(⊥) = ⊥ (can't compute successor of undefined)
- add(⊥, 5) = ⊥ (can't add undefined)

But some functions are **non-strict**:
- K x ⊥ = x (K ignores its second argument)
- if true then 5 else ⊥ = 5 (only evaluates the true branch)

In [ ]:
# Strictness in our dataset
print("Functions and their strictness:")
print()
print(functions_df[['function_name', 'preserves_bottom', 'notes']].head(10).to_string(index=False))

## 7. Function Spaces

If D and E are domains, then [D → E] (continuous functions from D to E) is also a domain!

The ordering on functions:
$$f \sqsubseteq g \iff \forall x. f(x) \sqsubseteq g(x)$$

**Pointwise ordering**: f is below g if f gives less information at every point.

The bottom function is λx.⊥ — undefined everywhere.

In [ ]:
# Function domain elements
from IPython.display import display

func_domain = functions_df[functions_df['domain'] == 'function_domain']

print("Function domain [D → E]:")
print()
for _, row in func_domain.iterrows():
    print(f"  {row['function_name']}: {row['notation']}")
    print(f"      {row['notes']}")
    print()

## 8. Composition Preserves Continuity

If f and g are continuous, so is f ∘ g.

**Proof**: 
```
(f ∘ g)(⊔D) = f(g(⊔D))
            = f(⊔{g(d) | d ∈ D})    (g continuous)
            = ⊔{f(g(d)) | d ∈ D}    (f continuous)
            = ⊔{(f ∘ g)(d) | d ∈ D}
```

This is why we can compose continuous functions freely.

In [ ]:
# Composition
print("Composition of continuous functions:")
print()
print("  If f: D → E and g: E → F are continuous,")
print("  then (g ∘ f): D → F is continuous.")
print()
print("  Example: double = λn. succ(succ(n))")
print("  Composition of successor with itself.")
print("  Both are continuous, so double is continuous.")

## Summary

In this tutorial, we learned:

1. **Monotonicity**: more info in → more info out
2. **Scott continuity**: f(⊔D) = ⊔{f(d) | d ∈ D}
3. **Continuous = computable**: only needs finite information
4. **Parallel or** is not continuous (needs simultaneous evaluation)
5. **Strictness**: f(⊥) = ⊥ for strict functions
6. **Function spaces** [D → E] are domains
7. **Composition** preserves continuity

In the next tutorial, we'll see the key theorem: continuous functions have **fixed points**.

---

## Exercises

1. Is the function λn. if n=0 then 1 else ⊥ continuous? Why?

2. Why can't parallel or be implemented on a sequential computer?

3. Give an example of a monotone function that is not continuous.

4. Is λf.f(f(0)) continuous as a function on [ℕ⊥ → ℕ⊥]?

---

*Next: Tutorial 5 - The Least Fixed Point*